In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.models import model_from_json

In [2]:
def eye_detect(json_file,weights_file):
    with open(json_file,"r") as file:
        loaded_model_json = file.read()
        model = model_from_json(loaded_model_json)

    model.load_weights(weights_file)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    return model

In [3]:
face_cascade = cv2.CascadeClassifier( 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier( 'haarcascade_eye_default.xml')
model = eye_detect('model_eye.json','model.h5')

In [7]:
cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces =  face_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3)
    eyes =   eye_cascade.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=3)
    
    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color = (255,0,0),thickness=3)
        
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color = (255,0,0),thickness=3)
         #preprocessing steps
        eye = frame[ey:ey+eh,ex:ex+ew]
        eye = cv2.resize(eye,(80,80))
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye,axis=0)
        
        # model prediction
        prediction = model.predict(eye)
        if prediction[0][0]>0.50:
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1)
            
        elif prediction[0][1]>0.50:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1)
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xff==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 30ms/step



KeyboardInterrupt

